# Zweifeldträger Schulbeispiel

Das Vorlesungsbeispiel verfolgt das Ziel, die Verformungen eines Zweifeldträgers für zwei Betonstähle mit unterschiedlichen Duktilitätsklassen aufzuzeigen. @jager_stahlbeton_2009.

![Statisches System des Zweifeldträgers mit Abmessungen versehen, daneben der Querschnitt mit Angaben zu der Längsbewehrung](../imgs/zweifeld_system.png){#fig-zweifeld_system}

Zunächst wird die analytische Lösung vollumfänglich dargestellt und mit numerischen Werten substituiert. Abschliessend wird das Verformungsverhalten mit dem Federmodell nachmodelliert, bzw. numerisch approximiert.

## Annahmen

- Zwei unterschiedliche Stähle
- konstant gerissene Biegesteifigkeit entlang der Stabachse

In [1]:
from packages import *

In [2]:
diam_x = 14 * mm  # Längsbewehrung in X-Richtung gerichtet
s_x = 200 * mm  # Teilung der Längsbewehrung
diam_y = 12 * mm  # Längsbewehrung in Y-Richtung gerichtet
s_y = 200 * mm  # Teilung der Längsbewehrung

c = 20 * mm  # Betonüberdeckung
h = 300 * mm  # Plattenstärke
b_w = 1 * m  # Streifenbreite
l = 8*m # Balkenlänge

d_x = h - c - diam_y - 1 / 2 * diam_x  # Statische Höhe in X-Richtung

put_out()

$$
\begin{aligned}
\oslash_{x} & = 14 \ \mathrm{mm} \quad & s_{x} & = 200 \ \mathrm{mm} \quad & \oslash_{y} & = 12 \ \mathrm{mm} \\ 
s_{y} & = 200 \ \mathrm{mm} \quad & c & = 20 \ \mathrm{mm} \quad & h & = 300 \ \mathrm{mm} \\ 
b_{w} & = 1 \ \mathrm{m} \quad & l & = 8 \ \mathrm{m} \quad & d_{x} & = 261.0 \ \mathrm{mm} \end{aligned}
$$

## Kaltverformter Betonstahl B500A

### Eigenschaften des Betonstahls


In [3]:
f_sy = 570 * N / mm**2  # Fliessgrenze
f_su = 600 * N / mm**2  # Zugfestigkeit
varepsilon_sy = 2.78*permille  # Fliessdehnung
varepsilon_su = 25*permille  # Bruchdehnung
E_s = 205 * kN / mm**2  # Elastizitätsmodul
E_sh = 1350 * N / mm**2  # Verfestigungsmodul

put_out()

$$
\begin{aligned}
f_{sy} & = 570.0 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \quad & f_{su} & = 600.0 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \quad & \varepsilon_{sy} & = 2.78 \ \mathrm{‰} \\ 
\varepsilon_{su} & = 25 \ \mathrm{‰} \quad & E_{s} & = 205.0 \ \frac{\mathrm{kN}}{\mathrm{mm}^{2}} \quad & E_{sh} & = 1350.0 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \end{aligned}
$$

In [4]:
# | echo: false

stress_b500a_array = np.array([0, f_sy.magnitude, f_su.magnitude]) * MPa
strain_b500a_array = np.array([0, varepsilon_sy, varepsilon_su])


fig, ax = plt.subplots()

ax.plot(strain_b500a_array, stress_b500a_array.to(MPa).magnitude, color="k")


# Set axis labels
ax.set_ylabel(r"$\sigma$ [N/mm$^2$]")
ax.set_xlabel(r"$\varepsilon_s$ [-]")

# Set limits
ax.set_xlim(xmax=125 / 1000)
ax.set_ylim(ymax=750)

# Show the plot
plt.tight_layout()
# plt.show()
plt.savefig("imgs/stress_strain_b500a.svg")
plt.close()

![Spannungs-Dehnungs-Beziehung des Betonstahls B500A](../imgs/stress_strain_b500a.svg){#stress_strain_b500a}

### Eigenschaften des Betons

In [5]:
# Zylinderdruckfestigkeit vorausgesetzt
f_cc = 30 * N / mm**2

# Betonzugfestigkeit, nicht einheitentreu
f_ct = (0.3 * f_cc.magnitude ** (2 / 3)) * N / mm**2 

# Effektive Betondruckfestigkeit, nicht einheitentreu
f_c = (2.5 * f_cc.magnitude ** (2 / 3)) * N / mm**2 

# Elastizitätsmodul des Betons
E_c = 29.3*kN/mm**2

put_out()

$$
\begin{aligned}
f_{cc} & = 30.0 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \quad & f_{ct} & = 2.9 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \quad & f_{c} & = 24.14 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \\ 
E_{c} & = 29.3 \ \frac{\mathrm{kN}}{\mathrm{mm}^{2}} \quad &  \quad &  
 \end{aligned}
$$

### Linear elastische - gerissene Biegesteifigkeit

Querschnittsanalyse:


In [6]:
# Querschnittsfläche der Längsbewehrung, pro Längeneinheit
a_s = (diam_x**2 / 4 * np.pi / s_x).to(mm**2 / m)

# Wertigkeit der Elastizitätsmodule
n = E_s / E_c

put_out()

$$
\begin{aligned}
a_{s} & = 769.69 \ \frac{\mathrm{mm}^{2}}{\mathrm{m}} \quad & n & = 7.0 \  \end{aligned}
$$

Die Druckzonenhöhe wird mittels dem Gleichgewicht der horizontalen Kräfte ermittelt

In [7]:
# Mittels Sympy (CAS für Python) wird eine analytische Lösung der
# Druckzonenhöhe für den gerissenen Querschnitt ermittelt.

# Summe der horizontalen Kräfte
sum_Fx = sp.sympify("1/2*x*b_w*sigma_c -sigma_s*a_s*b_w")

# Definition der Spannung in Abhängigkeit der Dehnungen
expr_sigma_c = sp.sympify("varepsilon_c * E_c")
expr_sigma_s = sp.sympify("varepsilon_s * n * E_c")

# Die Beziehung zwischen der Stahl- und Betondehnung nach Hook
expr_epsilon_c = sp.sympify("varepsilon_s /(d-x)*x")

# Substitution der Gleichungen
sum_Fx_subs = sum_Fx.subs(
    {
        "sigma_c": expr_sigma_c.subs("varepsilon_c", expr_epsilon_c),
        "sigma_s": expr_sigma_s,
    }
)

# Erstellen einer numerischen Funktion
x_solve = sp.solve(sum_Fx_subs, "x")[1]
x_solve_lambd = sp.lambdify(("a_s", "b_w", "n", "d"), x_solve)

In [8]:
# Druckzonenhöhe mit Parametern substituiert
x_II = x_solve_lambd(a_s, b_w, n, d_x).to(mm)

# Flächenmoment 2. Grades für den gerissenen Querschnitt
I_cII = (b_w * x_II**3 / 12 + b_w * x_II * (x_II / 2) ** 2).to(mm**4)
I_sII = (a_s * b_w * n * (d_x - x_II) ** 2).to(mm**4)
I_II = (I_cII + I_sII).to(mm**4)

# Gerissene Biegesteifigkeit
EI_II = (E_c * I_II).to(kN * m**2)

put_out()

$$
\begin{aligned}
x_{II} & = 47.91 \ \mathrm{mm} \quad & I_{cII} & = 36650532.72 \ \mathrm{mm}^{4} \quad & I_{sII} & = 244534349.46 \ \mathrm{mm}^{4} \\ 
I_{II} & = 281184882.18 \ \mathrm{mm}^{4} \quad & EI_{II} & = 8238.72 \ \mathrm{kN} \cdot \mathrm{m}^{2} \quad &  
 \end{aligned}
$$

Damit ist die Biegesteifigkeit definiert.

### Hebelarm der inneren Kräfte

Der Hebelarm der inneren Kräfte wird für den gesamten Träger als konstant angenommen.
Dabei wird dem Beton die Zylinderdruckfestigkeit hinterlegt, sowie erreich der Stahl die Zugfestigkeit.

In [9]:
# Druckzonenhöhe für den Zustand
x_dv = (a_s * b_w * f_su / (0.85 * f_cc * b_w)).to(mm)  

# Hebelarm der inneren Kräfte für diesen Zustand
d_v = d_x - 1 / 2 * 0.85 * x_dv  

# Biegewiderstand für diesen Zustand
m_us = (a_s * f_su * d_v).to(kN)

put_out()

$$
\begin{aligned}
x_{dv} & = 18.11 \ \mathrm{mm} \quad & d_{v} & = 253.3 \ \mathrm{mm} \quad & m_{us} & = 116.98 \ \mathrm{kN} \end{aligned}
$$

### Rissabstand

Frage an Hda, was hier gerechnet wird

In [10]:
# Abschätzung der mitwirkenden Höhe der Betonzugzone
h_eff = 2 * (h - d_x)  

# Geometrischer Bewehrungsgehalt
rho = (a_s / h_eff).to(percent)  

# maximaler Rissabstand
s_r0 = (diam_x * (1 - rho)  / (4 * rho)).to(mm)  

# minimaler Rissabstand
s_rl = 1/2 * s_r0 

# Annahme des Rissabstandes, entspricht dem Abstand der Querstäbe
s_r = 200 * mm  

put_out()

$$
\begin{aligned}
h_{eff} & = 78.0 \ \mathrm{mm} \quad & \rho & = 0.99 \ \mathrm{\%} \quad & s_{r0} & = 351.19 \ \mathrm{mm} \\ 
s_{rl} & = 175.59 \ \mathrm{mm} \quad & s_{r} & = 200 \ \mathrm{mm} \quad &  
 \end{aligned}
$$

### Risselement bei Fliessbeginn 

In [11]:
# Verbundschubspannung vor dem Fliessen
tau_b0 = 2 * f_ct

# minimale Stahlspannung
sigma_sy_3 = f_sy - 4 * tau_b0 * s_r / 2 / diam_x

# mittlere Stahldehnung
varepsilon_smy = ((sigma_sy_3 + f_sy) / (E_s * 2)).to(permille)

# Fliessmoment
m_y = (a_s * f_sy * d_v).to(kN*m/m)

put_out()

$$
\begin{aligned}
\tau_{b0} & = 5.79 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \quad & \sigma_{sy 3} & = 404.49 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \quad & \varepsilon_{smy} & = 2.38 \ \mathrm{‰} \\ 
m_{y} & = 111.13 \ \mathrm{kN} \quad &  \quad &  
 \end{aligned}
$$

### Risselement beim Erreichen der Zugfestigkeit

In [12]:
# Verbundschubspannung vor dem Fliessen
tau_b1 = f_ct

# Plastischer Bereich
Delta_x_pl = (f_su - f_sy) / (4*tau_b1 / diam_x)

# Elastischer Bereich
Delta_x_el = s_r /2 - Delta_x_pl

# minimale Stahlspannung
sigma_su_3 = f_sy - 4 * tau_b0 * Delta_x_el  / diam_x

put_out()

$$
\begin{aligned}
\tau_{b1} & = 2.9 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \quad & \Delta_{x pl} & = 36.25 \ \mathrm{mm} \quad & \Delta_{x el} & = 63.75 \ \mathrm{mm} \\ 
\sigma_{su 3} & = 464.49 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \quad &  \quad &  
 \end{aligned}
$$

In [13]:
# mittlere Stahldehnung
varepsilon_s_1 = varepsilon_su
varepsilon_s_2 = varepsilon_sy
varepsilon_s_3 = (sigma_su_3 / E_s).to(permille)

def center_trapez(a, b, d):
    return (a**2 - b**2 + abs(d) * (a + 2 * b)) / (3 * (a + b))

def area_trapez(a, b, h):
    return 1 / 2 * (a + b) * h

# Schwerpunkte der beiden Trapeze
S_1 = center_trapez(varepsilon_s_1, varepsilon_s_2, varepsilon_s_2)
S_2 = center_trapez(varepsilon_s_2, varepsilon_s_3, varepsilon_s_3)

# Flächen der beiden Trapeze
A_1 = area_trapez(varepsilon_s_1, varepsilon_s_2, Delta_x_pl)
A_2 = area_trapez(varepsilon_s_2, varepsilon_s_3, Delta_x_el)

varepsilon_sm = (S_1 * A_1 + S_2 * A_2) / (A_1 + A_2)

put_out()

$$
\begin{aligned}
\varepsilon_{s 1} & = 25 \ \mathrm{‰} \quad & \varepsilon_{s 2} & = 2.78 \ \mathrm{‰} \quad & \varepsilon_{s 3} & = 2.27 \ \mathrm{‰} \\ 
S_{1} & = 8.43 \ \mathrm{‰} \quad & S_{2} & = 1.27 \ \mathrm{‰} \quad & A_{1} & = 503.53 \ \mathrm{mm} \cdot \mathrm{‰} \\ 
A_{2} & = 160.83 \ \mathrm{mm} \cdot \mathrm{‰} \quad & \varepsilon_{sm} & = 6.69 \ \mathrm{‰} \quad &  
 \end{aligned}
$$

### Plastische Rotationswinkel


In [14]:
# Annahme für die Länge des plastischen Gelenks
l_pl = 2 * d_x 

# Plastischer Rotationswinkel, Rotationsbedarf für vollständige Umlagerung.
Theta_pl = (l_pl * (varepsilon_sm - varepsilon_smy) / (d_x - x_dv)).to(deg)

put_out()

$$
\begin{aligned}
l_{pl} & = 522.0 \ \mathrm{mm} \quad & \Theta_{pl} & = 0.53 \ \mathrm{°} \end{aligned}
$$

### Fliessen des Mittelauflagers

In [15]:
# Die Streckenlast zum Erreichen des Fliessens bei B
q_y_comma_B = 8 * m_y * b_w / l**2

# maximale Verformung im Feld
w_m_y = (5*q_y_comma_B*l**4/(384*EI_II) - m_y*b_w * l**2 / (16*EI_II)).to(mm)

put_out()

$$
\begin{aligned}
q_{y , B} & = 13.89 \ \frac{\mathrm{kN}}{\mathrm{m}} \quad & w_{m y} & = 35.97 \ \mathrm{mm} \end{aligned}
$$

### Fliessbeginn im Feld

In [16]:
# Streckenlast beim Fliessbeginn im Feld
q_y_comma_F = ((4 * m_y + 2 * m_us)/ l**2 + 4/l**2 * np.sqrt(m_y**2 + m_y *m_us))*b_w

put_out()

$$
\begin{aligned}
q_{y , F} & = 20.55 \ \frac{\mathrm{kN}}{\mathrm{m}} \end{aligned}
$$

#### Rotationsvermögen

In [17]:
# Rotationswinkel (Verwendung der Arbeitsgleichung)
alpha = ((q_y_comma_F - q_y_comma_B)* l**3 / (24*EI_II) - (m_us - m_y)*b_w*l/(3*EI_II)).to(deg)

# Verformungsbedarf
Theta_erf = 2 * alpha

#Verformungsvermögen
Theta_pl

put_out()

$$
\begin{aligned}
\alpha & = 0.88 \ \mathrm{°} \quad & \Theta_{erf} & = 1.76 \ \mathrm{°} \quad & \Theta_{pl} & = 0.53 \ \mathrm{°} \end{aligned}
$$

Das Verformungsvermögen ist nicht ausreichend.

In [18]:
# maximaler Rotationswinkel
alpha_max = Theta_pl / 2

# Streckenlast bis zum maximalen Rotationswinkel
Delta_qu = ((alpha_max + (m_us - m_y)*l*b_w / (3*EI_II))*24*EI_II/l**3).to(kN/m)

# Traglast für B500A
q_u_B500A = Delta_qu + q_y_comma_B

put_out()

$$
\begin{aligned}
\alpha_{max} & = 0.27 \ \mathrm{°} \quad & \Delta_{qu} & = 2.52 \ \frac{\mathrm{kN}}{\mathrm{m}} \quad & q_{u B500A} & = 16.41 \ \frac{\mathrm{kN}}{\mathrm{m}} \end{aligned}
$$

In [19]:
w_m_u = (5*q_u_B500A*l**4/(384*EI_II) - m_us*b_w * l**2 / (16*EI_II)).to(mm)    # maximale Verformung im Feld


put_out()

$$
\begin{aligned}
w_{m u} & = 49.46 \ \mathrm{mm} \end{aligned}
$$

## Naturharter Stahl B500C

### Eigenschaften des Betonstahls

In [20]:
f_sy = 500 * N / mm**2  # Fliessgrenze
f_su = 600 * N / mm**2  # Zugfestigkeit
varepsilon_sy = 2.439*permille  # Fliessdehnung
varepsilon_sh = 20*permille # Dehnung bei Verfestigungsbeginn
varepsilon_su = 125*permille  # Bruchdehnung
E_s = 205 * kN / mm**2  # Elastizitätsmodul
E_sh = 952 * N / mm**2  # Verfestigungsmodul

put_out()

$$
\begin{aligned}
f_{sy} & = 500.0 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \quad & f_{su} & = 600.0 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \quad & \varepsilon_{sy} & = 2.44 \ \mathrm{‰} \\ 
\varepsilon_{sh} & = 20 \ \mathrm{‰} \quad & \varepsilon_{su} & = 125 \ \mathrm{‰} \quad & E_{s} & = 205.0 \ \frac{\mathrm{kN}}{\mathrm{mm}^{2}} \\ 
E_{sh} & = 952.0 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \quad &  \quad &  
 \end{aligned}
$$

### Versagensart des Querschnitts

In [21]:
# Stahlspannung zwischen den Rissen
sigma_su_3 = f_su - 4 * tau_b1 / diam_x*s_r /2  


put_out()

$$
\begin{aligned}
\sigma_{su 3} & = 517.24 \ \frac{\mathrm{N}}{\mathrm{mm}^{2}} \end{aligned}
$$